# 개요

베이지안옵티마이저 (xgboost) 코드



# 1. 파일 업로드 및 기본 세팅

In [ ]:

# AIDU Library Import
from aicentro.session import Session
from aicentro.framework.keras import Keras as AiduFrm
aidu_session = Session(verify=False)
aidu_framework = AiduFrm(session=aidu_session)

# 데이터 파일 경로
data_path = aidu_framework.config.data_dir

import subprocess
import sys

import warnings
# 불필요한 경고 출력을 방지합니다.
warnings.filterwarnings('ignore')


#필요한 모델 다운로드
subprocess.call([sys.executable,"-m","pip","install","xgboost"])
subprocess.call([sys.executable,"-m","pip","install","seaborn"])
subprocess.call([sys.executable,"-m","pip","install","lightgbm"])
subprocess.call([sys.executable,"-m","pip","install","catboost"])
subprocess.call([sys.executable,"-m","pip","install","bayesian-optimization"])


# 2. 데이터 불러오기 및 기본 처리


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd


#Train file 데이터 디렉토리
train_file_dir = data_path + "/episode1_train.csv"
train_ob_file_dir = data_path + "/episode1_train_ob.csv"

#Test file 데이터 디렉토리
test_file_dir = data_path + "/episode1_test.csv"
test_ob_file_dir = data_path + "/episode1_test_ob.csv"


#전처리 후 test 파일명
test_file_name =data_path + "/pre_data/test_preprocessing.csv"

#전처리 후 train 파일명
train_file_name = data_path + "/pre_data/train_preprocessing.csv"

# 최적화 파라미터 결과 파일명
xgboost_param_file_name =  data_path + "/fit/optimizer/xgboost.csv"


train_df = pd.read_csv(train_file_name)
test_df = pd.read_csv(test_file_name)



In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
#학습하기위한 train과 validation 나눔
# 일반 데이터
x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'],random_state=21)


### 모델 학습 후 결과 시각화

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

my_predictions = {}

colors = ['r', 'c', 'm', 'y', 'k', 'khaki', 'teal', 'orchid', 'sandybrown',
          'greenyellow', 'dodgerblue', 'deepskyblue', 'rosybrown', 'firebrick',
          'deeppink', 'crimson', 'salmon', 'darkred', 'olivedrab', 'olive', 
          'forestgreen', 'royalblue', 'indigo', 'navy', 'mediumpurple', 'chocolate',
          'gold', 'darkorange', 'seagreen', 'turquoise', 'steelblue', 'slategray', 
          'peru', 'midnightblue', 'slateblue', 'dimgray', 'cadetblue', 'tomato'
         ]


#acc를 구해서 시각화해줌
def acc_eval(name_, pred, actual):
    global predictions
    global colors

    acc = (pred==actual).mean()
    my_predictions[name_] = acc


    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=False)  # 정확도 내림차순으로 sort
    
    df = pd.DataFrame(y_value, columns=['model', 'acc'])
    print(df)
    min_ = df['acc'].min() -1
    max_ = 1.2
    
    length = len(df)
    
    plt.figure(figsize=(10, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=15)
    bars = ax.barh(np.arange(len(df)), df['acc'])
    
    for i, v in enumerate(df['acc']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v+0.1, i, str(round(v, 3)), color='k', fontsize=15, fontweight='bold')
        
    plt.title('Accuracy', fontsize=18)
    plt.xlim(min_,max_)
    
    plt.show()
    
#acc를 구해서 시각화해줌(cross_val_score 사용하는 경우)
def acc_eval_add(name_, acc):
    global predictions
    global colors

    my_predictions[name_] = acc


    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=False)  # 정확도 내림차순으로 sort
    
    df = pd.DataFrame(y_value, columns=['model', 'acc'])
    print(df)
    min_ = df['acc'].min() -1
    max_ = 1.2
    
    length = len(df)
    
    plt.figure(figsize=(10, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=15)
    bars = ax.barh(np.arange(len(df)), df['acc'])
    
    for i, v in enumerate(df['acc']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v+0.1, i, str(round(v, 3)), color='k', fontsize=15, fontweight='bold')
        
    plt.title('Accuracy', fontsize=18)
    plt.xlim(min_,max_)
    
    plt.show()
    
    df.to_csv(acc_data_dir,index=False)
    

# 베이지안 옵티마이즈

## 1. XGBoost 베이지안옵티마이즈
https://wooono.tistory.com/102

In [ ]:
#사용하기 위한 import 

from sklearn.model_selection import cross_val_score
import numpy as np
import time
from sklearn.model_selection import KFold
from bayes_opt import BayesianOptimization

import xgboost as xgb
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import time


from sklearn.metrics import recall_score, precision_score, accuracy_score

# 베이지안 옵티마이저 시작 

In [ ]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, r2_score

def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100



def XGB_cv(max_depth,learning_rate, n_estimators, gamma, min_child_weight,max_delta_step,subsample,colsample_bytree,silent=True, nthread=-1):
    model = xgb.XGBClassifier(max_depth=int(max_depth),
                              learning_rate=learning_rate,
                              n_estimators=int(n_estimators),
                              nthread=nthread,
                              gamma=gamma,
                              min_child_weight=min_child_weight,
                              max_delta_step=max_delta_step,
                              subsample=subsample,
                              colsample_bytree=colsample_bytree,
                              use_label_encoder=False, eval_metric='logloss')
    
    
    eval_set = [(x_valid, y_valid)]

#     모델 훈련
    model.fit(x_train, y_train,  early_stopping_rounds=5, eval_metric="logloss", eval_set=eval_set, verbose=0)

#     예측값 출력
    y_pred= model.predict(x_valid)
    
# metric 계산
    acc = accuracy_score(y_valid,y_pred)
    return acc


In [ ]:
# 주어진 범위 사이에서 적절한 값을 찾는다.
pbounds = {'max_depth': (5, 10),
          'learning_rate': (0.01, 0.3),
          'n_estimators': (50, 300),
          'gamma': (1., 0.01),
          'min_child_weight': (2, 10),
          'max_delta_step': (0, 0.1),
          'subsample': (0.7, 0.9),
          'colsample_bytree' :(0.5, 0.99)          
          }

In [ ]:
# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제
xgboostBO = BayesianOptimization(f = XGB_cv,pbounds = pbounds, verbose = 2, random_state = 1 )


#출력하기
out_df = pd.DataFrame(xgboostBO.max)

opti_result_file_name = data_path +'/fit/optimizer/optimizer.csv'
out_df['rows']= out_df.index
out_df.to_csv(opti_result_file_name, header=True, index=True, encoding='utf-8')



In [ ]:
# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)
import time

start = time.time()

xgboostBO.maximize(init_points=4, n_iter = 20, acq='ei', xi=0.01)
print('BO Time(min): {:.2f}'.format((time.time()-start)/60))

In [ ]:
# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(xgboostBO.max)

In [ ]:
#파라미터 결과 파일로 출력

print(dict((xgboostBO.max['params'])))
out_param =pd.DataFrame()

out_param['param']=list(dict((xgboostBO.max['params'])).keys())
out_param['value']=list(dict((xgboostBO.max['params'])).values())

out_param.to_csv(xgboost_param_file_name, header=True, index=True, encoding='utf-8'  )
print('최대 파라미터가 출력 되었습니다.',xgboost_param_file_name)

out_param

# 베이지안 결과로 나온 파라미터로 결과 예측

In [ ]:
#최대 결과로 예측값 
fit_xgb = xgb.XGBClassifier(max_depth= int( xgboostBO.max['params']['max_depth'] ),
                             learning_rate=xgboostBO.max['params']['learning_rate'],
                             n_estimators=int(xgboostBO.max['params']['n_estimators']),
                             gamma= xgboostBO.max['params']['gamma'],
                             min_child_weight=xgboostBO.max['params']['min_child_weight'],
                             max_delta_step=xgboostBO.max['params']['max_delta_step'],
                             subsample=xgboostBO.max['params']['subsample'],
                             colsample_bytree=xgboostBO.max['params']['colsample_bytree'],
                             n_jobs=-1)

fit_xgb.fit(x_train,y_train)
pred_XGB = fit_xgb.predict(x_valid)

#그래프 그리기
acc_eval("XGBOOST 최적화 후" ,pred_XGB, y_valid)